Based on:  
https://gist.github.com/IevaZarina/ef63197e089169a9ea9f3109058a9679
https://www.kdnuggets.com/2017/03/simple-xgboost-tutorial-iris-dataset.html

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
import joblib

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.datasets as datasets
from sklearn.metrics import (
    accuracy_score, 
    recall_score, 
    f1_score, 
    precision_score, 
    confusion_matrix
)
from sklearn.model_selection import train_test_split

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
print(X.shape)
print(y.shape)

In [ ]:
main_df = pd.DataFrame(iris.data, columns=iris.feature_names)
main_df['target'] = iris.target
main_df.info()
main_df.head(10)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations

In [ ]:
model = xgb.train(param, dtrain, num_round)

In [ ]:
model.dump_model('../models/iris_model_dump.txt')

In [ ]:
preds = model.predict(dtest)
print(preds.shape)

In [ ]:
best_preds = np.ascontiguousarray([np.argmax(line) for line in preds])
print(best_preds)

In [ ]:
def calculate_metrics(predicted, y_test):
    cm = confusion_matrix(y_test, predicted)
    acc = accuracy_score(y_test, predicted)
    precision = precision_score(y_test, predicted, average='macro')
    recall = recall_score(y_test, predicted, average='macro')
    f_score = f1_score(y_test, predicted, average='macro')
    print(f'Accuracy: {acc}\nPrecision: {precision}\nRecall: {recall}\nF1_score: {f_score}\n')
    return cm

In [ ]:
sns.set_theme()
sns.set_style('whitegrid')
plt.tight_layout()

In [ ]:
cm = calculate_metrics(best_preds, y_test)
sns.heatmap(cm, annot=True, fmt='g')
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
joblib.dump(model, '../models/iris_model.pkl', compress=True)